In [ ]:
%azure.connect "/subscriptions/REDACTED/internship-demo" location="westus"


In [ ]:
open Microsoft.Quantum.Arithmetic;
open Microsoft.Quantum.Arrays;
open Microsoft.Quantum.Convert;
open Microsoft.Quantum.Diagnostics;
open Microsoft.Quantum.Math;
open Microsoft.Quantum.Measurement;

In [ ]:
operation MeasureColor (register : Qubit[]) : Int {
    return MeasureInteger(LittleEndian(register));
}

In [ ]:
operation MeasureColoring (bitsPerColor : Int, register : Qubit[]) : Int[] {
    let numVertices = Length(register) / bitsPerColor;
    let colorPartitions = Partitioned(ConstantArray(numVertices - 1, bitsPerColor), register);
    return ForEach(MeasureColor, colorPartitions);
}

In [ ]:
operation ApplyColorEqualityOracle(
    color0 : Qubit[], color1 : Qubit[],
    target : Qubit
)
: Unit is Adj + Ctl {
    within {
        // compute XOR of q0 and q1 in place (storing it in q1).
        ApplyToEachCA(CNOT, Zipped(color0, color1));
    } apply {
        // if all XORs are 0, the bit strings are equal.
        ControlledOnInt(0, X)(color1, target);
    }
}

operation ApplyVertexColoringOracle (
    numVertices : Int, bitsPerColor : Int, edges : (Int, Int)[],
    startingColorConstraints : (Int, Int)[],
    colorsRegister : Qubit[],
    target : Qubit
)
: Unit is Adj + Ctl {
    let nEdges = Length(edges);

    let nStartingColorConstraints = Length(startingColorConstraints);
    // we are looking for a solution that:
    // (a) has no edge with same color at both ends and
    // (b) has no Vertex with a color that violates the starting color constraints.
    use edgeConflictQubits = Qubit[nEdges];
    use startingColorConflictQubits = Qubit[nStartingColorConstraints];
    within {
        ConstrainByEdgeAndStartingColors(
            colorsRegister, edges, startingColorConstraints,
            edgeConflictQubits, startingColorConflictQubits, bitsPerColor
        );
    } apply {
        // If there are no conflicts (all qubits are in 0 state), the vertex coloring is valid.
        ControlledOnInt(0, X)(edgeConflictQubits + startingColorConflictQubits, target);
    }
}

operation ConstrainByEdgeAndStartingColors(
    colorsRegister : Qubit[], edges : (Int, Int)[],
    startingColorConstraints : (Int, Int)[],
    edgeConflictQubits : Qubit[], startingColorConflictQubits : Qubit[], bitsPerColor: Int
)
: Unit is Adj + Ctl {
    for ((start, end), conflictQubit) in Zipped(edges, edgeConflictQubits) {
        // Check that endpoints of the edge have different colors:
        // apply ColorEqualityOracle_Nbit oracle;
        // if the colors are the same the result will be 1, indicating a conflict
        ApplyColorEqualityOracle(
            colorsRegister[start * bitsPerColor .. (start + 1) * bitsPerColor - 1],
            colorsRegister[end * bitsPerColor .. (end + 1) * bitsPerColor - 1],
            conflictQubit
        );
    }
    for ((cell, value), conflictQubit) in Zipped(startingColorConstraints, startingColorConflictQubits) {
        // Check that cell does not clash with starting colors.
        ControlledOnInt(value, X)(
            colorsRegister[cell * bitsPerColor .. (cell + 1) * bitsPerColor - 1],
            conflictQubit
        );
    }

}

In [ ]:

operation ApplyVertexColoringOracle4Bit9Color (numVertices : Int, edges : (Int, Int)[],
    startingColorConstraints : (Int, Int)[],
    colorsRegister : Qubit[], target : Qubit) : Unit is Adj+Ctl {
    let nEdges = Length(edges);
    let bitsPerColor = 4; // 4 bits per color
    let nStartingColorConstraints = Length(startingColorConstraints);
    // we are looking for a solution that:
    // (a) has no edge with same color at both ends and
    // (b) has no Vertex with a color that violates the starting color constraints.
    use edgeConflictQubits = Qubit[nEdges];
    use startingColorConflictQubits = Qubit[nStartingColorConstraints];
    use vertexColorConflictQubits = Qubit[numVertices];
    within {
        ConstrainByEdgeAndStartingColors(
            colorsRegister, edges, startingColorConstraints,
            edgeConflictQubits, startingColorConflictQubits, bitsPerColor
        );
        let zippedColorAndConfictQubit = Zipped(
            Partitioned(ConstantArray(numVertices, bitsPerColor), colorsRegister),
            vertexColorConflictQubits
        );
        for (color, conflictQubit) in zippedColorAndConfictQubit {
            // Only allow colors from 0 to 8 i.e. if bit #3 = 1, then bits 2..0 must be 000.
            use tempQubit = Qubit();
            within {
                ApplyOrOracle(color[0 .. 2], tempQubit);
            } apply{
                // AND color's most significant bit with OR of least significant bits.
                // This will set conflictQubit to 1 if color > 8.
                CCNOT(color[3], tempQubit, conflictQubit);
            }
        }
    } apply {
        // If there are no conflicts (all qubits are in 0 state), the vertex coloring is valid.
        ControlledOnInt(0, X)(edgeConflictQubits + startingColorConflictQubits + vertexColorConflictQubits, target);
    }
}

operation ApplyOrOracle (queryRegister : Qubit[], target : Qubit) : Unit is Adj {
    // x₀ ∨ x₁ = ¬ (¬x₀ ∧ ¬x₁)
    // First, flip target if both qubits are in |0⟩ state.
    ControlledOnInt(0, X)(queryRegister, target);
    // Then flip target again to get negation.
    X(target);
}

operation FindColorsWithGrover (numVertices : Int, bitsPerColor : Int, maxIterations : Int,
    oracle : ((Qubit[], Qubit) => Unit is Adj)) : Int[] {
    // This task is similar to task 2.2 from SolveSATWithGrover kata,
    // but the percentage of correct solutions is potentially higher.
    mutable coloring = new Int[numVertices];

    // Note that coloring register has the number of qubits that is
    // twice the number of vertices (bitsPerColor qubits per vertex).
    use register = Qubit[bitsPerColor * numVertices];
    use output = Qubit();

    mutable correct = false;
    mutable iter = 1;
    // Try for one iteration, if it fails, try again for one more iteration and repeat until maxIterations is reached.
    repeat {
        Message($"Trying search with {iter} iterations...");
        ApplyGroversAlgorithmLoop(register, oracle, iter);
        let res = MultiM(register);
        // to check whether the result is correct, apply the oracle to the
        // register plus auxiliary after measurement.
        oracle(register, output);
        if (MResetZ(output) == One) {
            set correct = true;
            // Read off coloring.
            set coloring = MeasureColoring(bitsPerColor, register);
        }
        ResetAll(register);
    } until (correct or iter > maxIterations)
    fixup {
        set iter += 1;
    }
    if (not correct) {
        fail "Failed to find a coloring.";
    }

    return coloring;
}

operation ApplyPhaseOracle (oracle : ((Qubit[], Qubit) => Unit is Adj),
    register : Qubit[]
)
: Unit is Adj {
    use target = Qubit();
    within {
        // Put the target into the |-⟩ state.
        X(target);
        H(target);
    } apply {
        // Apply the marking oracle; since the target is in the |-⟩ state,
        // flipping the target if the register satisfies the oracle condition
        // will apply a -1 factor to the state.
        oracle(register, target);
    }
    // We put the target back into |0⟩ so we can return it.
}

operation ApplyGroversAlgorithmLoop(
    register : Qubit[],
    oracle : ((Qubit[], Qubit) => Unit is Adj),
    iterations : Int
)
: Unit {
    let applyPhaseOracle = ApplyPhaseOracle(oracle, _);
    ApplyToEach(H, register);

    for _ in 1 .. iterations {
        applyPhaseOracle(register);
        within {
            ApplyToEachA(H, register);
            ApplyToEachA(X, register);
        } apply {
            Controlled Z(Most(register), Tail(register));
        }
    }
}
